# Tähelepanumehhanismid ja transformerid

Üks peamisi korduvvõrkude puudusi on see, et kõik sõnad järjestuses mõjutavad tulemust võrdselt. See põhjustab standardsete LSTM kodeerija-dekodeerija mudelite puhul alamoptimaalset jõudlust järjestuste vaheliste ülesannete, nagu nimede tuvastamine ja masintõlge, lahendamisel. Tegelikkuses on teatud sisendjärjestuse sõnadel sageli suurem mõju järjestikustele väljunditele kui teistel.

Vaatleme järjestuse-järjestuse mudelit, näiteks masintõlget. See on rakendatud kahe korduvvõrgu abil, kus üks võrk (**kodeerija**) koondab sisendjärjestuse varjatud olekusse ja teine, **dekodeerija**, lahtirullib selle varjatud oleku tõlgitud tulemuseks. Selle lähenemise probleem seisneb selles, et võrgu lõplikul olekul on raske meeles pidada lause algust, mis põhjustab mudeli kehva kvaliteeti pikkade lausete puhul.

**Tähelepanumehhanismid** pakuvad võimalust kaaluda iga sisendvektori kontekstuaalset mõju RNN-i iga väljundprognoosi puhul. Selle rakendamine toimub lühenduste loomisega sisend-RNN-i vaheolekute ja väljund-RNN-i vahel. Sel viisil, kui genereerime väljundisümbolit $y_t$, võtame arvesse kõiki sisendi varjatud olekuid $h_i$, erinevate kaalukoefitsientidega $\alpha_{t,i}$.

![Pilt, mis näitab kodeerija/dekodeerija mudelit koos aditiivse tähelepanukihiga](../../../../../translated_images/et/encoder-decoder-attention.7a726296894fb567.webp)
*Kodeerija-dekodeerija mudel koos aditiivse tähelepanumehhanismiga [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), viidatud [selles blogipostituses](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Tähelepanumaatriks $\{\alpha_{i,j}\}$ esindab, millisel määral teatud sisendsõnad osalevad antud sõna genereerimisel väljundjärjestuses. Allpool on näide sellisest maatriksist:

![Pilt, mis näitab näidisalini, mille leidis RNNsearch-50, võetud Bahdanau - arviz.org](../../../../../translated_images/et/bahdanau-fig3.09ba2d37f202a6af.webp)

*Joonis võetud [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Joonis 3)*

Tähelepanumehhanismid vastutavad suure osa praeguse või peaaegu praeguse tipptaseme eest loomuliku keele töötlemises. Tähelepanu lisamine suurendab aga oluliselt mudeli parameetrite arvu, mis põhjustas RNN-ide skaleerimisprobleeme. RNN-ide skaleerimise peamine piirang on see, et mudelite korduv olemus muudab treeningu rühmitamise ja paralleelimise keeruliseks. RNN-is tuleb järjestuse iga element töödelda järjestikuses järjekorras, mis tähendab, et seda ei saa lihtsalt paralleelselt töödelda.

Tähelepanumehhanismide kasutuselevõtt koos selle piiranguga viis tänapäeval tuntud ja kasutatavate tipptasemel transformerimudelite loomiseni, nagu BERT ja OpenGPT3.

## Transformerimudelid

Selle asemel, et edastada iga eelneva prognoosi konteksti järgmisse hindamissammu, kasutavad **transformerimudelid** **positsioonikodeeringuid** ja **tähelepanu**, et haarata antud sisendi konteksti etteantud tekstiaknas. Allolev pilt näitab, kuidas positsioonikodeeringud koos tähelepanuga suudavad konteksti haarata antud aknas.

![Animeeritud GIF, mis näitab, kuidas hindamisi tehakse transformerimudelites.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Kuna iga sisendpositsioon kaardistatakse sõltumatult iga väljundpositsiooniga, suudavad transformerid paremini paralleelselt töötada kui RNN-id, mis võimaldab palju suuremaid ja väljendusrikkamaid keelemudeleid. Iga tähelepanukeskus saab kasutada erinevate sõnadevaheliste seoste õppimiseks, mis parandab loomuliku keele töötlemise ülesandeid.

## Lihtsa transformerimudeli loomine

Keras ei sisalda sisseehitatud transformeri kihti, kuid me saame selle ise ehitada. Nagu varem, keskendume AG News andmestiku tekstiklassifikatsioonile, kuid tasub mainida, et transformerimudelid näitavad parimaid tulemusi keerukamate NLP ülesannete puhul.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Uued kihid Kerases peaksid pärinema `Layer` klassist ja rakendama `call` meetodit. Alustame **Positsioonilise Embeddingu** kihiga. Kasutame [mõningast koodi ametlikust Kerase dokumentatsioonist](https://keras.io/examples/nlp/text_classification_with_transformer/). Eeldame, et täidame kõik sisendjärjestused pikkuseni `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

See kiht koosneb kahest `Embedding` kihist: üks on mõeldud tokenite sisestamiseks (nagu me varem arutasime) ja teine tokenite positsioonide jaoks. Tokenite positsioonid luuakse naturaalarvude järjestusena vahemikus 0 kuni `maxlen`, kasutades `tf.range`, ning seejärel edastatakse need sisestuskihile. Kaks saadud sisestusvektorit liidetakse, mille tulemusena saadakse positsiooniliselt sisestatud sisendi esitus kujuga `maxlen`$\times$`embed_dim`.

<img src="../../../../../translated_images/et/pos-embedding.e41ce9b6cf6078af.webp" width="40%"/>

Nüüd rakendame transformer ploki. See võtab sisendiks varem määratletud sisestuskihi väljundi:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer rakendab `MultiHeadAttention` positsiooniliselt kodeeritud sisendile, et luua tähelepanuvektor mõõtmetega `maxlen`$\times$`embed_dim`, mis seejärel segatakse sisendiga ja normaliseeritakse `LayerNormalization` abil.

> **Note**: `LayerNormalization` on sarnane `BatchNormalization`-iga, mida käsitleti *Arvutinägemise* osas selle õppeprogrammi raames, kuid see normaliseerib eelmise kihi väljundid iga treeningnäidise jaoks eraldi, viies need vahemikku [-1..1].

Selle kihi väljund edastatakse seejärel läbi `Dense` võrgu (meie puhul - kahekihi perceptron), ja tulemus lisatakse lõplikule väljundile (mis läbib uuesti normaliseerimise).

<img src="../../../../../translated_images/et/transformer-layer.905e14747ca4e7d5.webp" width="30%" />

Nüüd oleme valmis defineerima täieliku transformer mudeli:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Mudelid

**BERT** (Bidirectional Encoder Representations from Transformers) on väga suur mitmekihiline transformer-võrk, millel on 12 kihti *BERT-base* jaoks ja 24 kihti *BERT-large* jaoks. Mudel treenitakse esmalt suurtel tekstikorpustel (WikiPedia + raamatud) kasutades juhendamata treenimist (ennustades lauses maskeeritud sõnu). Eeltreeningu käigus omandab mudel märkimisväärse taseme keele mõistmist, mida saab seejärel kasutada koos teiste andmekogumitega peenhäälestuse abil. Seda protsessi nimetatakse **ülekandeõppeks**.

![pilt aadressilt http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/et/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformer-arhitektuuridel, sealhulgas BERT, DistilBERT, BigBird, OpenGPT3 ja teistel, on palju variatsioone, mida saab peenhäälestada.

Vaatame, kuidas saame kasutada eeltreenitud BERT-mudelit, et lahendada meie traditsioonilist järjestuse klassifitseerimise probleemi. Laename idee ja osa koodist [ametlikust dokumentatsioonist](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Eeltreenitud mudelite laadimiseks kasutame **Tensorflow hub**-i. Kõigepealt laadime BERT-spetsiifilise vektoriseerija:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Oluline on kasutada sama vektoriseerijat, millega algne võrk treeniti. Lisaks tagastab BERT vektoriseerija kolm komponenti:
* `input_word_ids`, mis on sisendlause jaoks mõeldud tokenite numbrite järjestus
* `input_mask`, mis näitab, milline osa järjestusest sisaldab tegelikku sisendit ja milline osa on täitmine. See on sarnane maskile, mida genereerib `Masking` kiht
* `input_type_ids` kasutatakse keelemodelleerimise ülesannete jaoks ja võimaldab määrata kaks sisendlause ühes järjestuses.

Seejärel saame instantsida BERT-i omaduste ekstraktori:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Niisiis, BERT-i kiht tagastab mitmeid kasulikke tulemusi:
* `pooled_output` on tulemuseks kõigi järjestuse tokenite keskmistamine. Seda võib vaadelda kui kogu võrgu intelligentset semantilist esindust. See on samaväärne meie eelmise mudeli `GlobalAveragePooling1D` kihi väljundiga.
* `sequence_output` on viimase transformer-kihi väljund (vastab ülaltoodud mudeli `TransformerBlock` väljundile).
* `encoder_outputs` on kõigi transformer-kihtide väljundid. Kuna oleme laadinud 4-kihilise BERT-mudeli (nagu võib arvata nimest, mis sisaldab `4_H`), on sellel 4 tensori. Viimane neist on sama, mis `sequence_output`.

Nüüd määratleme otsast lõpuni klassifitseerimismudeli. Kasutame *funktsionaalset mudeli määratlust*, kus määratleme mudeli sisendi ja seejärel esitame rea avaldisi selle väljundi arvutamiseks. Samuti muudame BERT-mudeli kaalud mitte-treenitavaks ja treenime ainult lõplikku klassifikaatorit:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Kuigi treenitavaid parameetreid on vähe, on protsess üsna aeglane, sest BERT-i omaduste ekstraktor on arvutuslikult raske. Tundub, et me ei suutnud saavutada mõistlikku täpsust, kas treeningu puudumise või mudeli parameetrite vähesuse tõttu.

Proovime BERT-i kaalu lahti lukustada ja seda samuti treenida. See nõuab väga väikest õppemäära ning ka hoolikamat treeningstrateegiat koos **soojendusega**, kasutades **AdamW** optimeerijat. Kasutame optimeerija loomiseks `tf-models-official` paketti:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Nagu näete, toimub treening üsna aeglaselt – kuid võite katsetada ja treenida mudelit mõne epohhi (5–10) jooksul, et näha, kas saate parema tulemuse võrreldes varasemate lähenemistega.

## Huggingface Transformers'i teek

Teine väga levinud (ja veidi lihtsam) viis Transformer-mudelite kasutamiseks on [HuggingFace pakett](https://github.com/huggingface/), mis pakub lihtsaid ehitusplokke erinevate NLP ülesannete jaoks. See on saadaval nii Tensorflow'le kui ka PyTorchile, mis on teine väga populaarne tehisnärvivõrkude raamistik.

> **Note**: Kui te ei soovi näha, kuidas Transformers'i teek töötab, võite liikuda otse selle märkmiku lõppu, kuna te ei näe midagi oluliselt erinevat sellest, mida me eespool tegime. Me kordame samu BERT-mudeli treenimise samme, kasutades teistsugust teeki ja oluliselt suuremat mudelit. Seetõttu hõlmab protsess üsna pikka treeningut, nii et võite lihtsalt koodi läbi vaadata.

Vaatame, kuidas meie probleemi saab lahendada, kasutades [Huggingface Transformers](http://huggingface.co).


Esimene asi, mida peame tegema, on valida mudel, mida hakkame kasutama. Lisaks mõnedele sisseehitatud mudelitele sisaldab Huggingface [veebipõhist mudelite repositooriumi](https://huggingface.co/models), kus kogukond on jaganud palju rohkem eeltreenitud mudeleid. Kõiki neid mudeleid saab laadida ja kasutada lihtsalt mudeli nime esitamisega. Kõik vajalikud binaarfailid mudeli jaoks laaditakse automaatselt alla.

Mõnikord võib tekkida vajadus laadida oma mudelid, sel juhul saate määrata kataloogi, mis sisaldab kõiki asjakohaseid faile, sealhulgas tokeniseerija parameetreid, `config.json` faili mudeli parameetritega, binaarkaale jne.

Mudeli nime põhjal saame luua nii mudeli kui ka tokeniseerija. Alustame tokeniseerijast:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` objekt sisaldab `encode` funktsiooni, mida saab otse kasutada teksti kodeerimiseks:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Me saame kasutada ka tokenizerit, et kodeerida järjestust viisil, mis sobib mudelile edastamiseks, st sisaldades `token_ids`, `input_mask` välju jne. Samuti saame määrata, et soovime Tensorflow tensoreid, lisades argumendi `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Meie puhul kasutame eelnevalt treenitud BERT-mudelit nimega `bert-base-uncased`. *Uncased* tähendab, et mudel ei erista suuri ja väikeseid tähti.

Mudeli treenimisel peame sisendiks andma tokeniseeritud järjestuse, seega loome andmetöötluse torujuhtme. Kuna `tokenizer.encode` on Pythoni funktsioon, kasutame sama lähenemist nagu eelmises osas, kutsudes seda `py_function` abil:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nüüd saame laadida tegeliku mudeli, kasutades `BertForSequenceClassification` paketti. See tagab, et meie mudelil on juba vajalik klassifikatsiooni arhitektuur, sealhulgas lõplik klassifikaator. Näete hoiatusteadet, mis ütleb, et lõpliku klassifikaatori kaalud ei ole algväärtustatud ja mudel vajab eelõpetamist - see on täiesti normaalne, sest just seda me kavatseme teha!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Nagu näete `summary()`-st, sisaldab mudel peaaegu 110 miljonit parameetrit! Tõenäoliselt, kui soovime lihtsat klassifitseerimisülesannet suhteliselt väikese andmestikuga, ei soovi me treenida BERT-i põhikihti:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nüüd oleme valmis alustama treenimist!

> **Märkus**: Täismahus BERT-mudeli treenimine võib olla väga aeganõudev! Seetõttu treenime seda ainult esimese 32 partii jaoks. See on lihtsalt selleks, et näidata, kuidas mudeli treenimine on üles seatud. Kui soovite proovida täismahus treenimist - eemaldage lihtsalt `steps_per_epoch` ja `validation_steps` parameetrid ning olge valmis ootama!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Kui suurendate iteratsioonide arvu, ootate piisavalt kaua ja treenite mitu epohhi, võite eeldada, et BERT klassifikatsioon annab parima täpsuse! Seda seetõttu, et BERT mõistab keele struktuuri juba üsna hästi ja meil on vaja ainult lõplikku klassifikaatorit peenhäälestada. Kuid kuna BERT on suur mudel, võtab kogu treenimisprotsess kaua aega ja nõuab tõsist arvutusvõimsust! (GPU ja eelistatavalt rohkem kui üks).

> **Note:** Meie näites oleme kasutanud üht väikseimat eeltreenitud BERT mudelit. On olemas suuremaid mudeleid, mis tõenäoliselt annavad paremaid tulemusi.


## Peamine mõte

Selles peatükis tutvusime väga hiljutiste mudeliarhitektuuridega, mis põhinevad **transformeritel**. Kasutasime neid tekstiklassifikatsiooni ülesande jaoks, kuid samamoodi saab BERT-mudeleid kasutada üksuste tuvastamiseks, küsimustele vastamiseks ja muude NLP ülesannete lahendamiseks.

Transformer-mudelid esindavad praegust tipptaset NLP-s ning enamasti peaksid need olema esimene lahendus, millega alustad katsetamist, kui rakendad kohandatud NLP-lahendusi. Siiski on äärmiselt oluline mõista korduvate närvivõrkude põhimõtteid, mida selles moodulis käsitleti, kui soovid luua keerukaid närvimudeleid.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
